In [1]:
import os
import os.path
import argparse
import torch
import torch.optim as optim
from models import resunet
from utils.runner import Trainer
from config import MASSdataset

In [2]:
class ResUNet(object):
    def __init__(self,args):
        self.model = resunet.ResUNet(args.in_ch,args.out_ch,args.base_kernel,args.is_leaky)
        
def main(args):
    method = os.path.basename(os.getcwd()).split(".")[0]
    if args.is_leaky:
        method +='-leaky'
    if args.cuda and not torch.cuda.is_available():
        raise ValueError("GPUs are not available,please run at cpu mode")
    
    #initialize datasets
    datasets = MASSdataset()
    
    args.in_ch = datasets.in_ch
    args.out_ch = datasets.out_ch
    #initialize network
    net = ResUNet(args)
    
    if args.cuda:
        net.model.cuda()
    net.optimizer=optim.Adam(net.model.parameters(),lr=args.lr,betas=optim_betas)
    #initializer runner
    run = Trainer(args,method,is_multi=False)
    print("Start trainning {}...".format(method))
    with torch.no_grad():
        run.training(net,[datasets.train,datasets.val])
        run.save_log()
        run.learning_curve()
    
        run.evaluating(net.model,datasets.train,'train')
        run.evaluating(net.model,datasets.validation,'val')
        run.evaluating(net.model,datasets.test,'test')
        run.save_checkpoint(net.model)

    
if __name__=="__main__":
    parser = argparse.ArgumentParser(description='ArgumentParser')
    parser.add_argument('-is_leaky',type=bool,default=False,
                        help='used leakyRELU or not')
    parser.add_argument("-trigger",type=str,default='epoch',choices=['epoch', 'iter'],
                        help='trigger type for logging')
    parser.add_argument('-interval',type=int,default=10,
                       help='interval for logging')
    parser.add_argument('-terminal',type=int,default=100,
                       help='terminal for training')
    parser.add_argument('-base_kernel',type=int,default=24,
                       help='base number of kernels')
    parser.add_argument('-batch_size',type=int,default=32,
                       help='batch_size for training')
    parser.add_argument('-lr',type=float,default=2e-4,
                       help='learning rate for discriminator')
    parser.add_argument('-cuda',type=lambda x:(str(x).lower=='true'),default=True,
                       help='using cuda for optimization')
    args= parser.parse_args(args=[])
    optim_betas = (0.9,0.999)
    main(args)

Start trainning DL...


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [3]:
x=torch.FloatTensor([1,2,3])

In [8]:
from torch.autograd import Variable
x=Variable(x,required_grad=True)

TypeError: 'required_grad' is an invalid keyword argument for this function